# Problem 2: The Forward Pass of a Neural Network

_Version 1.1_

Neural networks are a set of algorithms for pattern recognition. They are loosely inspired by the human brain, and have proven especially useful in data clustering and classification tasks.

In this problem, you will use your knowledge of Python and Numpy to speed-up a common computational kernel that arises in neural networks.

**Example: a "fully connected layer" for images.** Suppose the data we are analyzing consists of $N$ two-dimensional images of size $H \times W$ pixels each. In a neural network, a typical substep is the evaluation of a "fully connected (FC) layer," which takes the images as input and produces a vector of outputs, with one vector per image.

Mathematically, here is a simplified example of what a typical FC layer calculation might look like. Let $x[k, i, j]$ denote the value (e.g., intensity) of the pixel at location $(i, j)$ of the $k$-th input image. Since there are $N$ images, take the values of $k$ to be in the range of 0 to $N-1$, respectively. And since each image is $H \times W$, take $0 \leq i \leq H-1$ and $0 \leq j \leq W-1$. Next, let $\mathrm{out}[k, l]$ denote an output value in the $l$-th element of a vector associated with image $k$, where $0 \leq l \leq M-1$, for some given value of $M$. Lastly, suppose the specific  formula for transforming the input images into this collection of output vectors is given by the formula,

$$
\mathrm{out}[k, l] = b[l] + \sum_{i=0}^{H-1} \sum_{j=0}^{W-1} \left(x[k, i, j] \times w[l, i, j]\right)
$$

where $w[l, i, j]$ are "weights" and $b[l]$ are "biases." The process of "training" the neural network from sample data determines these weight and bias parameters, but for this problem, just assume that they are given.

If it's helpful, here is a picture of what this formula is doing for each $(k, l)$ pair:
<img src = "fully_connected.png" width = "600">

**The baseline implementation.** In the code cells below, we define a Python function, `FC_naive(x, w, b)`, that implements the FC layer calculation from above using a straightforward, albeit somewhat naive, method. Your goal is to make this baseline run faster.

To start, first run the next three code cells to estimate the time of the baseline implementation.

In [1]:
import numpy as np
import time

def rel_error(x, y):
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [2]:
def FC_naive(x, w, b):
    """
    Inputs:
    - x: A numpy array of images of shape (N, H, W)
    - w: A numpy array of weights of shape (M, H, W)
    - b: A numpy vector of biases of size M

    Returns: 
    - out: a numpy array of shape (N, M)
    """
    N, H, W = x.shape
    M, _, _ = w.shape
    out = np.zeros((N,M))
    for ni in range(N):
        for mi in range(M):
                out[ni,mi] = b[mi]
                for d1 in range(H):
                    for d2 in range(W):
                        out[ni,mi] += x[ni, d1, d2] * w[mi, d1, d2] 
    return out

In [3]:
num_inputs = 50
input_shape = (128, 256)
output_dim = 10

x = np.random.rand(num_inputs, *input_shape)
w = np.random.rand(output_dim, *input_shape)
b = np.random.rand(output_dim)
start_time = time.time ()
out = FC_naive(x, w, b)
elapsed_time = time.time () - start_time
print ("==> Took %g seconds." % elapsed_time)

==> Took 9.56739 seconds.


**Exercise 1** (5 points). Let's start by seeing if we can make `FC_naive()` function faster by rewriting the two innermost loops, i.e., the `d1` and `d2` loops:

```python
for d1 in range(H):
    for d2 in range(W):
        out[ni, mi] += x[ni, d1, d2] * w[mi, d1, d2]
```

For this exercise, complete the function `two_inner_loops(x_i, w_l, b_j)`, below, so that it implements the same computation as these two `d1` and `d2` loops, but is much faster. It should return `out[ni, mi]`. The input `x_i` is the `i`-th image, `w_l` is the `l`-th weight matrix, and `b_l` is the `l`-th component of the bias vector.

The test code will check your results and benchmark a complete FC layer using the function `FC_two_loops()`, defined below. You'll see that it calls your `two_inner_loops()` routine to implement the two innermost loops.

To get credit on this exercise, the resulting execution time of `FC_two_loops()` must be at least **100 times faster** than `FC_naive()` on the problem sizes being tested below when running on the Vocareum platform. There is no partial credit for smaller speedups. Having said that, a little bit of basic Numpy should go a long way.

In [4]:
def two_inner_loops(x_i, w_l, b_l):
    """
    Inputs:
    - x_i: A numpy array of images of shape (H, W)
    - w_l: A numpy array of weights of shape (H, W)
    - b_l: A float (single number)

    Returns: 
    - out: A float (single number)
    """
    
    #ravel function flatens the array. We need a single float output by 
    #summing multiplicaiton of respective elements of arrays and 
    out = x_i.ravel().dot(w_l.ravel()) + b_l
    
    return out


In [5]:
# Test cell: 'FC_two_loops_1' (5 points)

def FC_two_loops(x, w, b):
    """
    Inputs:
    - x: A numpy array of images of shape (N, H, W)
    - w: A numpy array of weights of shape (M, H, W)
    - b: A numpy vector of biases of size M

    Returns: 
    - out: a numpy array of shape (N, M)
    """
    N, H, W = x.shape
    M, _, _ = w.shape
    out = np.zeros((N,M))
    for ni in range(N):
           for mi in range(M):
                out[ni, mi] = two_inner_loops(x[ni,  :, :], w[mi,  :, :], b[mi])
    return out

num_inputs = 50
input_shape = (128, 256)
output_dim = 10

x = np.random.rand(num_inputs, *input_shape)
w = np.random.rand(output_dim, *input_shape)
b = np.random.rand(output_dim)

start_time = time.time ()
for i in range(5):
    out_naive = FC_naive(x, w, b)
elapsed_time_naive = (time.time () - start_time)/5
print ("==> Took %g seconds." % elapsed_time_naive)

start_time = time.time ()
for i in range(5):
    out_fast = FC_two_loops(x, w, b)
elapsed_time_fast = (time.time () - start_time)/5
print ("==> Took %g seconds." % elapsed_time_fast)

error = rel_error(out_naive, out_fast)
print("Output error:", error)
assert error < 1e-12, "The value of your output is incorrect or not accurate enough"

speed_up = elapsed_time_naive/elapsed_time_fast
print("Speed-up:", speed_up)
assert speed_up > 100, "The speed-up of your method is less than 100"

print("\n(Passed!)")

==> Took 9.08344 seconds.
==> Took 0.00593128 seconds.
Output error: 6.421729158587902e-15
Speed-up: 1531.4454850950256

(Passed!)


**Question 2** (5 points). Now, completely rewrite the `FC_naive()` function by at least **1,000 times**.

> This improvement can be attained with basic Numpy and no explicit loops.

In [59]:
def FC_no_loop(x, w, b):
    """
    Inputs:
    - x: A numpy array of images of shape (N, H, W)
    - w: A numpy array of weights of shape (M, H, W)
    - b: A numpy vector of biases of size M

    Returns: 
    - out: a numpy array of shape (N, M)
    """
    N, H, W = x.shape
    M, _, _ = w.shape
    out = np.zeros((N,M))
    
    #Einsum explained -> https://stackoverflow.com/questions/26089893/understanding-numpys-einsum
    #out = np.einsum('nhw,mhw->nm', x, w)
    #out = out + b
    #einsum gives correct answer but for some reason einsum did not speed things up, was only producing 830x run-time improvement.

    for ni in range(N):
           for mi in range(M):
                out[ni, mi] = (x[ni,  :, :].ravel().dot(w[mi,  :, :].ravel())) + b[mi]
    return out    

In [60]:
num_inputs = 2
input_shape = (1, 1)
output_dim = 2

x = np.random.rand(num_inputs, *input_shape)
w = np.random.rand(output_dim, *input_shape)
b = np.random.rand(output_dim)
start_time = time.time ()

out1 = FC_two_loops(x, w, b)
out2 = FC_no_loop(x, w, b)

print (b.shape,'\n',b,'\n')

print (x.shape,'\n',x,'\n')


print (w.shape,'\n',w,'\n')

print (FC_naive(x, w, b).shape,'\n',FC_naive(x, w, b))

print (FC_two_loops(x, w, b).shape,'\n',FC_two_loops(x, w, b))

print (FC_no_loop(x, w, b).shape,'\n',FC_no_loop(x, w, b))


(2,) 
 [0.56722924 0.86718359] 

(2, 1, 1) 
 [[[0.18291467]]

 [[0.07557256]]] 

(2, 1, 1) 
 [[[0.3681297 ]]

 [[0.76284718]]] 

(2, 2) 
 [[0.63456557 1.00671953]
 [0.59504975 0.9248339 ]]
(2, 2) 
 [[0.63456557 1.00671953]
 [0.59504975 0.9248339 ]]
(2, 2) 
 [[0.63456557 1.00671953]
 [0.59504975 0.9248339 ]]


In [61]:
# Test cell: 'FC_no_loop' (5 points)
num_inputs = 50
input_shape = (128, 256)
output_dim = 10

x = np.random.rand(num_inputs, *input_shape)
w = np.random.rand(output_dim, *input_shape)
b = np.random.rand(output_dim)

start_time = time.time ()
for i in range(5):
    out_naive = FC_naive(x, w, b)
elapsed_time_naive = (time.time () - start_time)/5
print ("==> Took %g seconds." % elapsed_time_naive)

start_time = time.time ()
for i in range(5):
    out_fast = FC_no_loop(x, w, b)
elapsed_time_fast = (time.time () - start_time)/5
print ("==> Took %g seconds." % elapsed_time_fast)

error = rel_error(out_naive, out_fast)
print("Output error:", error)
assert error < 1e-12, "The value of your output is incorrect or not accurate enough"

speed_up = elapsed_time_naive/elapsed_time_fast
print("Speed-up:", speed_up)
assert speed_up > 1000, "The speed-up of your method is less than 1000"

print("\n(Passed!)")

==> Took 10.7717 seconds.
==> Took 0.0101097 seconds.
Output error: 6.766913946038391e-15
Speed-up: 1065.4757518300505

(Passed!)


**Fin!** You've reached the end of this problem. Don't forget to restart the
kernel and run the entire notebook from top-to-bottom to make sure you did
everything correctly. If that is working, try submitting this problem. (Recall
that you *must* submit and pass the autograder to get credit for your work!)